In [1]:
# -
#
# SPDX-FileCopyrightText: Copyright (c) 2024 Pietro Carlo Boldini, Rene Pecnik and the CUBENS contributors. All rights reserved.
# SPDX-License-Identifier: MIT
#
# -

import math as m
import cmath as cm
import numpy as np
import scipy.optimize as opt
from scipy.interpolate import interp1d
from numpy import linalg as npla

from scipy.sparse.linalg import spsolve
from scipy.sparse        import diags, hstack, vstack
from scipy.integrate     import solve_bvp


from functools import partial
import CoolProp.CoolProp as CP

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rcParams.update({'font.size': 18})
rcParams['figure.figsize']   = [8,6]
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family']      = 'STIXGeneral'

# Main class for CHA calculation

In [2]:
class CHA():
  
    def writeToFile(self):
        file = open("inputDNS/initCHA_params.h", "w")
        file.write("!------------ use equation of state ------------------ \n")
        file.write('USE_EOS  = "PR" \n')
        file.write("!------------ set non-dimensional reference values for computation ------------------ \n")
        file.write("Pra   = %.10f \n" % self.Pr)
        file.write("Ec   = %.10e \n" % self.Ec)
        file.write("Ma   = %.10e \n" % self.Ma)
        file.write("eos_dof   = %.6f \n" % self.dof)
        file.write("eos_ac   = %.6f \n" % self.omega_ac)
        file.write("!------------ set dimensional values for computation ------------------ \n")
        file.write("Tcrit   = %.6f \n" % self.T_crit)
        file.write("Pcrit   = %.6f \n" % self.P_crit)
        file.write("Vcrit   = %.6e \n" % self.V_crit)
        file.write("eos_Rgas   = %.6f \n" % self.Rg)
        file.write("! ------------ set wall BC for computation ------------------ \n")
        file.write("Twall_bot   = %.6f \n" % self.Twall_bot)
        file.write("Twall_top   = %.6f \n" % self.Twall_top)
        file.write("!------------ set non-dimensional reference values for computation ------------------ \n")
        file.write("Tref   = %.10f \n" % self.Tred)
        file.write("Pref   = %.10f \n" % self.Pred)
        file.write("Rhoref   = %.10f \n" % self.Rhored)
        file.write("Cpref   = %.10f \n" % self.CpR)
        file.write("SOSref   = %.10f \n" % self.sos) 
        file.write("Rref   = %.10f \n" % self.R) 
        file.write("! ----------- set viscosity and conductivity ---------------- \n")
        file.write('USE_VISC  = "%s" \n' % self.visc_bc)
        if ((self.visc_bc == 'JST') or (self.visc_bc == 'Chung')) :
            file.write("Muref   = %.10e \n" % self.mu_inf)
            file.write("Kref   = %.10e \n" % self.ka_inf)
        file.close()
        
    def showParameters(self):
        print("\nDNS parameters:\n")
        print("USE_EOS = PR ")
        print("Pra = ", self.Pr)
        print("Ec = ", self.Ec)
        print("Ma = ", self.Ma)
        print("Pref = ", self.Pred)
        print("Cv/R= = ", self.CvR)
        print('USE_VISC  = ', self.visc_bc)
        
        print('\nDNS initial conditions are saved in ./inputDNS/')
        print('DNS parameters are saved in ./inputDNS/')

# Different EoS

## Peng-Robinson

In [3]:
class CHA_PR(CHA): 
    def __init__(self, Ec = None, Twall_bot=None, Twall_top=None, Pred =None, Tred =None, CvR=None, visc = None, Pr=None):   
        self.Ec = Ec  
        self.Twall_bot = Twall_bot
        self.Twall_top = Twall_top
        self.Pred = Pred
        self.Tred = Tred 
        self.CvR = CvR
        self.f_pr()
        rho0 = CP.PropsSI("D", "P", Pred*self.P_crit, "T", Tred*self.T_crit, self.fluid)/self.Rho_crit # this is just an approximation
        self.Rhored = float(opt.fsolve(self.f_rh,rho0,args=(Tred)))
        self.Hred = self.f_h(self.Rhored, self.Tred)
        self.CpR = self.f_cp(self.Rhored, self.Tred, self.CvR) 
        self.sos = self.f_sos(self.Rhored, self.Tred, self.CvR)
        self.Ma=(self.Ec*self.CpR*self.Tred/self.sos**2/self.Zc)**0.5
        self.dof = 9  
        if visc == 'Constant':
            self.visc_bc = 'Constant'
            self.mu_inf = self.Tred
            self.ka_inf = self.mu_inf
            self.Pr = Pr
        elif visc == 'JST':
            self.visc_bc = 'JST'
            [self.mu_inf,self.ka_inf] = self.f_muka(self.Rhored, self.Tred, visc)
            self.Pr = Pr
        elif visc == 'Chung':
            self.visc_bc = 'Chung'
            [self.mu_inf,self.ka_inf] = self.f_muka(self.Rhored, self.Tred, visc)
            self.Pr=self.CpR*self.mu_inf/self.ka_inf*self.Rg
    
    def f_pr(self):
        self.Zc=0.3112
        self.Zc_1=self.Zc**(-1)
        self.Zc_2=self.Zc**(-2)
        self.R=1/self.Zc
        self.a=0.45724
        self.b=0.07780
        self.Ru=8.31451
        self.fluid="CO2"
        self.omega_ac=0.224
        self.K=0.37464+1.54226*self.omega_ac-0.26992*self.omega_ac**2
        self.dof=9
        self.M=CP.PropsSI("molemass",self.fluid)
        self.Rg=self.Ru/self.M         
        self.T_crit=CP.PropsSI("Tcrit",self.fluid)
        self.P_crit=CP.PropsSI("Pcrit",self.fluid)
        self.Zc_RP=0.274586376 # exact Zc according to tables for the transport properties
        self.V_crit=self.Zc_RP*self.Rg*self.T_crit/self.P_crit
        self.Rho_crit=1/self.V_crit
        return self
    
    def f_chung(self):
        self.A_chung = 1.16145
        self.B_chung = 0.14874
        self.C_chung = 0.52487
        self.D_chung = 0.77320
        self.E_chung = 2.16178
        self.F_chung = 2.43787
        self.G_chung = -6.435 * 10**(-4)
        self.H_chung = 7.27371
        self.S_chung = 18.0323
        self.W_chung = -0.76830
        self.Fc = 1 - 0.2756 * self.omega_ac  # only for non-polar
        a0 = [6.32402, 0.0012102, 5.28346, 6.62263, 19.7454, -1.89992, 24.2745, 0.79716, -0.23816, 0.068629]
        a1 = [50.4119, -0.0011536, 254.209, 38.0957, 7.63034, -12.5367, 3.44945, 1.11764, 0.067695, 0.34793]
        self.Arho = [a0[i] + a1[i] * self.omega_ac for i in range(len(a0))]
        b0 = [2.41657, -0.50924, 6.61069, 14.54250, 0.79274, -5.86340, 81.171]
        b1 = [0.74824, -1.50936, 5.62073, -8.91387, 0.82019, 12.80050, 114.158]
        self.Brho = [b0[i] + b1[i] * self.omega_ac for i in range(len(b0))]
        self.alpha = self.dof / 2 - 3 / 2
        self.beta = 0.7862 - 0.7109 * self.omega_ac + 1.3168 * self.omega_ac**2  # only for non-polar
        return self
        
    def f_rh_T(self, x, h):
        T, rho = x
        F1 = T * self.Zc_1 / ((1/rho) - self.b * self.Zc_1) - self.a * ((1 + self.K * (1 - np.sqrt(T)))**2) * self.Zc_2 / ((1/rho) * ((1/rho) + 2 * self.b * self.Zc_1) - self.b**2 * self.Zc_2) - self.Pred
        F2 = (self.CvR * T * self.Zc_1 +
              self.a * self.Zc_1 / (2 * m.sqrt(2) * self.b) * ((self.K + 1)**2 - self.K * (self.K + 1) * np.sqrt(T)) *
              np.log((1 + self.b * self.Zc_1 / (1/rho) * (1 - m.sqrt(2))) / (1 + self.b * self.Zc_1 / (1/rho) * (1 + m.sqrt(2)))) +
              self.Pred * (1/rho) - h * self.Hred) 
        return np.array([F1, F2])
    
    def f_rh(self, rho, T):
        v = 1/rho
        alpha = (1 + self.K * (1 - np.sqrt(T)))**2
        Vol = T * self.Zc_1 / (v - self.b * self.Zc_1) - alpha * self.a * self.Zc_2 / (v * (v + 2 * self.b * self.Zc_1) - self.b**2 * self.Zc_2) - self.Pred
        return np.array(Vol)
    
    def f_h(self, rho, T):
        self.f_cp(rho, T, self.CvR)
        e_PR = self.CvR * T * self.Zc_1 + self.a * self.Zc_1 / (2 * m.sqrt(2) * self.b) * ((self.K + 1)**2 - self.K * (self.K + 1) * np.sqrt(T)) * np.log((1 + self.b * self.Zc_1 * rho * (1 - m.sqrt(2))) / (1 + self.b * self.Zc_1 * rho * (1 + m.sqrt(2))))
        return e_PR + self.Pred / rho
    
    def f_cp(self, rho, T, CvR):
        alpha = (1 + self.K * (1 - np.sqrt(T)))**2
        Cv_rho = self.CvR - (self.a * self.K * (self.K + 1)) / (4 * self.b * np.sqrt(2 * T)) * np.log((1 + self.b * self.Zc_1 * rho * (1 - m.sqrt(2))) / (1 + self.b * self.Zc_1 * rho * (1 + m.sqrt(2))))
        dPdT_Rho = self.K * np.sqrt(alpha / T) * (rho**2 * self.a * self.Zc_2) / (1 + 2 * self.b * self.Zc_1 * rho - self.b**2 * self.Zc_2 * rho**2) - (rho * self.Zc_1) / (rho * self.b * self.Zc_1 - 1)
        dPdRho_T = self.Zc_1 * T / (self.Zc_1 * self.b * rho - 1)**2 - self.a * self.Zc_2 * alpha * (2 * rho + 2 * self.b * self.Zc_1 * rho**2) / (1 + 2 * self.b * self.Zc_1 * rho - rho**2 * self.b**2 * self.Zc_2)**2
        return np.array(Cv_rho + T / rho**2 * self.Zc * (dPdT_Rho**2) / dPdRho_T)
    
    def f_sos(self, rho, T, CvR):
        alpha = (1 + self.K * (1 - np.sqrt(T)))**2
        Cv_rho = self.CvR - (self.a * self.K * (self.K + 1)) / (4 * self.b * np.sqrt(2 * T)) * np.log((1 + self.b * self.Zc_1 * rho * (1 - m.sqrt(2))) / (1 + self.b * self.Zc_1 * rho * (1 + m.sqrt(2))))
        dPdT_Rho = self.K * np.sqrt(alpha / T) * (rho**2 * self.a * self.Zc_2) / (1 + 2 * self.b * self.Zc_1 * rho - self.b**2 * self.Zc_2 * rho**2) - (rho * self.Zc_1) / (rho * self.b * self.Zc_1 - 1)
        dPdRho_T = self.Zc_1 * T / (self.Zc_1 * self.b * rho - 1)**2 - self.a * self.Zc_2 * alpha * (2 * rho + 2 * self.b * self.Zc_1 * rho**2) / (1 + 2 * self.b * self.Zc_1 * rho - rho**2 * self.b**2 * self.Zc_2)**2
        return (dPdRho_T + self.Zc * T / rho**2 / Cv_rho * dPdT_Rho**2)**0.5

    def f_muka(self, rho, T, visc):
        if visc == 'Constant':
            return np.maximum(T,1.0e-6), np.maximum(T,1.0e-6)
        elif visc == 'JST':
            # viscosity
            mu_1 = np.where(T <= 1.50,
                            34 * 10**(-5) * T**0.94,
                            17.78 * 10**(-5) * (4.58 * T - 1.67)**(5/8))
            f_rho = 0.10230 + 0.023364 * rho + 0.058533 * rho**2 - 0.040758 * rho**3 + 0.0093324 * rho**4
            mu_diff = (f_rho**4 - 10**(-4))
            # conductivity
            kappa_factor=(0.307*self.CvR+0.539)
            kappa_EU = 15 / 4 * self.Ru * mu_1 * kappa_factor
            kappa_RT_tri = 6.54 * 10**(-5) * (np.exp(0.2826 * T) - 1 / np.exp(0.3976 * T**2))
            kappa = np.zeros_like(rho)
            mask1 = rho < 0.50
            mask2 = (rho >= 0.50) & (rho < 2.0)
            mask3 = rho >= 2.0
            f_rho1 = 14.0 * (np.exp(0.535 * rho) - 1)
            kappa_diff1 = (f_rho1 * 10**(-8)) / self.Zc**5
            kappa[mask1] = kappa_diff1[mask1] * (4.1868 / 10**(-2)) + kappa_EU[mask1]
            f_rho2 = 13.1 * (np.exp(0.67 * rho) - 1.069)
            kappa_diff2 = (f_rho2 * 10**(-8)) / self.Zc**5
            kappa[mask2] = kappa_diff2[mask2] * (4.1868 / 10**(-2)) + kappa_EU[mask2]
            f_rho3 = 2.976 * (np.exp(1.155 * rho) + 2.016)
            kappa_diff3 = (f_rho3 * 10**(-8)) / self.Zc**5
            kappa[mask3] = kappa_diff3[mask3] * (4.1868 / 10**(-2)) + kappa_EU[mask3]
            return mu_diff + mu_1, kappa
        elif visc == 'Chung':
            self.f_chung()
            T_dim = T*self.T_crit
            # viscosity
            V_crit = self.V_crit * self.M * 10**6
            ek = self.T_crit / 1.2593
            T_dimless = T_dim / ek
            psi = (self.A_chung / T_dimless**self.B_chung) + (self.C_chung / np.exp(self.D_chung * T_dimless)) + (self.E_chung / np.exp(self.F_chung * T_dimless)) + (self.G_chung * T_dimless**self.B_chung * np.sin(self.S_chung * T_dimless**self.W_chung - self.H_chung))
            mu_0 = 4.0785 * 10**(-6) * self.Fc * (self.M * 10**3 * T_dim)**0.5 / (V_crit**(2/3) * psi)  # new version
            Y = rho * self.Zc_RP / self.Zc / 6
            G1 = (1 - 0.5 * Y) / (1 - Y)**3
            G2 = (self.Arho[0] * (1 - np.exp(-self.Arho[3] * Y)) / Y + self.Arho[1] * G1 * np.exp(self.Arho[4] * Y) + self.Arho[2] * G1) / (self.Arho[0] * self.Arho[3] + self.Arho[1] + self.Arho[2])
            mu_k = mu_0 * (1 / G2 + self.Arho[5] * Y)
            mu_p = 3.6344 * 10**(-6) * (self.M * 10**3 * self.T_crit)**0.5 / (V_crit**(2/3)) * (self.Arho[6] * Y**2 * G2 * np.exp(self.Arho[7] + self.Arho[8] * T_dimless**(-1) + self.Arho[9] * T_dimless**(-2)))
            # conductivity
            Z = 2.0 + 10.5 * T**2
            Psi = 1 + self.alpha * (0.215 + 0.28288 * self.alpha - 1.061 * self.beta + 0.26665 * Z) / (0.6366 + self.beta * Z + 1.061 * self.alpha * self.beta)
            kappa_0 = 31.2 * (mu_0 / self.M) * Psi
            H2 = (self.Brho[0] * (1 - np.exp(-self.Brho[3] * Y)) / Y + self.Brho[1] * G1 * np.exp(self.Brho[4] * Y) + self.Brho[2] * G1) / (self.Brho[0] * self.Brho[3] + self.Brho[1] + self.Brho[2])
            kappa_k = kappa_0 * (1 / H2 + self.Brho[5] * Y)
            kappa_p = 3.586 * 10**(-3) * (self.T_crit / self.M)**0.5 / (V_crit**(2/3)) * (self.Brho[6] * Y**2 * H2 * T**(0.5))
            return mu_k + mu_p, kappa_k + kappa_p         
    

# Solve channel with Peng-Robinson fluid

In [4]:
# Input parameters of CHA_PR:
# 1) Reference Eckert number (Mach number will be calculated): Ec = u^2_b/(Cp_ref T_ref)
# 2) Ratio: bottom wall temperature to reference temperature
# 3) Ratio: top wall temperature to reference temperature
# 4) Reference reduced pressure (p_r = p_ref/p_crit)
# 5) Reference reduced temperature  (T_r = T_ref/T_crit)
# 6) Cv/R - ratio
# 7) Viscosity law: 'Chung', 'JST' (JossiStielThodos), or 'Constant'
# 8) Prandtl number: if 'Chung', then it's newly calculated and overwritten, 
#                    if other viscosity laws, then it is kept.

pr = CHA_PR(Ec=0.004,Twall_bot=1.0,Twall_top=1.0,Pred=1.084,Tred=0.9207,CvR=9/2,visc='Chung',Pr=1.0)
pr.writeToFile()
pr.showParameters()



DNS parameters:

USE_EOS = PR 
Pra =  2.3947469378513917
Ec =  0.004
Ma =  0.1282811176952621
Pref =  1.084
Cv/R= =  4.5
USE_VISC  =  Chung

DNS initial conditions are saved in ../inputDNS/
DNS parameters are saved in ../inputDNS/
